# 1-Drift Detection and Test

In [4]:
import evidently

In [5]:
import pandas as pd
import numpy as np

In [24]:
df_t=pd.read_csv(r"..\data\df_i.csv")

df_t.head()

In [8]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [9]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
feat = ['Over_Drawn_MEAN', 'Missed_Payment_Date_MEAN',
       'AMT_DRAWINGS_ATM_CURRENT_MEAN', 'AMT_BALANCE_MEAN', 'DAYS_BIRTH',
       'CNT_DRAWINGS_CURRENT_MEAN', 'DAYS_CREDIT_ENDDATE_MEAN',
       'DAYS_REGISTRATION', 'DAYS_CREDIT_ENDDATE_MIN',
       'CREDIT_TYPE_Credit_card_MEAN']

In [38]:
data = df_t

feat = data.columns.tolist()
feat.remove('SK_ID_CURR')
feat.remove('TARGET')
feat

['DAYS_REGISTRATION',
 'FLAG_WORK_PHONE',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_BIRTH',
 'DAYS_INSTALMENT_MEAN',
 'CNT_INSTALMENT_MEAN',
 'DAYS_DECISION_MEAN',
 'NAME_CONTRACT_STATUS_Refused_MEAN',
 'NAME_YIELD_GROUP_high_MEAN']

In [40]:
reference = data[0:1000]#.sample(n=500, replace=False)
current = data[-1000:]#.sample(n=500, replace=False)

# 1-1 Drift detection

Michaël vous demande également de tester l’utilisation de la librairie evidently pour détecter dans le futur du Data Drift en production. Pour cela vous prendrez comme hypothèse que le dataset “application_train” représente les datas pour la modélisation et le dataset “application_test” représente les datas de nouveaux clients une fois le modèle en production. 

L’analyse à l’aide d’evidently vous permettra de détecter éventuellement du Data Drift sur les principales features, entre les datas d’entraînement et les datas de production, au travers du tableau HTML d’analyse que vous aurez réalisé.

In [ ]:
application_train
application_test

In [41]:
Report
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=reference, current_data=current)
report

# Test suite

In [48]:
tests = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
])

tests.run(reference_data=reference, current_data=current)
tests

In [52]:
suite = TestSuite(tests=[
    TestColumnDrift('NAME_YIELD_GROUP_high_MEAN'),
    TestMeanInNSigmas('NAME_CONTRACT_STATUS_Refused_MEAN'),
    NoTargetPerformanceTestPreset(columns=['DAYS_LAST_PHONE_CHANGE', 'DAYS_INSTALMENT_MEAN', 'FLAG_WORK_PHONE'])
])

suite.run(reference_data=reference, current_data=current)
suite

In [53]:
suite.save_html('test_suite.html')

In [51]:
suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
    TestColumnDrift('DAYS_LAST_PHONE_CHANGE'),
    TestShareOfOutRangeValues('DAYS_LAST_PHONE_CHANGE'),
    DataStabilityTestPreset(),
    RegressionTestPreset()
    
])

suite.run(reference_data=reference, current_data=current)
suite

ValueError: The columns 'target' and 'prediction' columns should be present